Let's make a pandas dataframe with columns.

Metadata
1. path
2. fps
2. position
3. object
5. t_duration
4. t_start
5. t_end
6. f_start
7. f_end
8. n_lasers

Now the actual data should have the columns:
1. time (30_000)
2. shifts_raw (100,30_000,2)
3. shifts (100,30_000,2)
4. fft freq (100,11_000,2)
5. fft val (100,11_000,2)
9. fft val phase-synced (100,11_000,2)
6. mode fft freq (100,5,2)
8. mode time (100,5,2)
7. mode fft val (100,5,2)
10. mode fft val phase-synced (100,5,2)
 
Visualizations:
1. x,y shifts over all lasers over time
2. FFT magnitude averaged over lasers, time
3. modes
    1. FFT magnitude
    2. x,y shifts
    3. surface reconstruction
3. Dimension Reduction:
    1. PCA 2D,3D + cumulative PCA variance + PCA pairwise distance
    2. Diffusion Map 2D,3D + cumulative Diffusion Map variance + Diffusion Map pairwise distance

compare 2D PCA of
1. (shifts, shifts_raw)
2. (fft val magnitude, fft val angle, fft val magnitude+angle)
3. (all frequencies, just mode frequencies)
4. (not phase-synced, phase-synced)
5. (duplicates, no duplicates)

The ideal results is that the best representation comes from doing PCA on:
1. phase-synced
2. modes
3. with magnitude+amplitude
4. and duplicates

I think I should use polars, hf_dataset?

Some metrics!
1. Measure the variance from 3 PCA
2. Measure the number of PCAs needed to reach 90% variance
3. Measure for what number of PCA the duplicates are closest to each other

Plot the number of modes VS the above metrics!

# 1. Setup

In [6]:
# reload all modules every time before executing
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from pathlib import Path
import sys, types

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [8]:
# Patch for pickle compatibility
if 'recover_core_lib' not in sys.modules:
    print('adding `recover_core_lib` to sys')
    fake = types.ModuleType('recover_core_lib')
    fake.compute_CAM2_translations_v3_cupy = lambda *a, **k: None
    sys.modules['recover_core_lib'] = fake

adding `recover_core_lib` to sys


In [9]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [10]:
%matplotlib widget

# 2. Load Data